# Module biogeme.optimization 

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the class. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2021-10-26 16:03:50.675276


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.9a [2021-10-26]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import numpy as np
import pandas as pd

In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [5]:
import biogeme.optimization as opt
import biogeme.biogeme as bio
import biogeme.database as db
import biogeme.models as models
from biogeme.expressions import Beta, Variable


Define the verbosity of Biogeme

In [6]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setSilent()
#logger.setDetailed()
#logger.setDebug()

# Biogeme example

In [7]:
df = pd.DataFrame({'Person': [1, 1, 1, 2, 2],
                   'Exclude': [0, 0, 1, 0, 1],
                   'Variable1': [1, 2, 3, 4, 5],
                   'Variable2': [10, 20, 30,40, 50],
                   'Choice': [1, 2, 3, 1, 2],
                   'Av1': [0, 1, 1, 1, 1],
                   'Av2': [1, 1, 1, 1, 1],
                   'Av3': [0, 1, 1, 1, 1]})
myData = db.Database('test', df)

Choice = Variable('Choice')
Variable1 = Variable('Variable1')
Variable2 = Variable('Variable2')
beta1 = Beta('beta1', 0, None, None, 0)
beta2 = Beta('beta2', 0, None, None, 0)
V1 = beta1 * Variable1
V2 = beta2 * Variable2
V3 = 0
V ={1: V1,2: V2,3: V3}

likelihood = models.loglogit(V, av=None, i=Choice)
myBiogeme = bio.BIOGEME(myData, likelihood)
myBiogeme.modelName = 'simpleExample'
myBiogeme.saveIterations = False
myBiogeme.generateHtml = False
myBiogeme.generatePickle = False
print(myBiogeme)

simpleExample: database [test]{'loglike': _bioLogLogitFullChoiceSet(1:(beta1(0) * Variable1), 2:(beta2(0) * Variable2), 3:`0`)}
simpleExample: database [test]{'loglike': _bioLogLogitFullChoiceSet(1:(beta1(0) * Variable1), 2:(beta2(0) * Variable2), 3:`0`)}


In [8]:
f, g, h, gdiff, hdiff = myBiogeme.checkDerivatives(verbose=True)

In [9]:
gdiff

array([6.03961326e-07, 6.11038797e-05])

In [10]:
hdiff

array([[ 8.31415072e-07, -4.16670705e-05],
       [-4.27475899e-06,  8.33270355e-04]])

## scipy

This is the optimization algorithm from scipy. It is possible to transfer parameters to the algorithm. We include here an irrelevant parameter to illustrate the warning. 

In [11]:
results = myBiogeme.estimate(algorithm=opt.scipy, 
                             algoParameters={'myparam':3})
results.getEstimatedParameters()

/Users/michelbierlaire/opt/anaconda3/envs/python39/lib/python3.9/site-packages/biogeme-3.2.9a0-py3.9-macosx-10.9-x86_64.egg/biogeme/optimization.py:74: OptimizeWarning: Unknown solver options: myparam
  results = sc.minimize(


,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.391017,0.369666,0.711632,0.366198,0.394720,0.693049
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685574,0.492982


In [12]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	scipy.optimize
Cause of termination:	CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Number of iterations:	13
Number of function evaluations:	16
Optimization time:	0:00:00.005071


## Newton with linesearch

In [13]:
results = myBiogeme.estimate(algorithm=opt.newtonLineSearchForBiogeme)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685573,0.492982


In [14]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with line search
Relative gradient:	3.43067908440321e-07
Number of iterations:	3
Number of function evaluations:	10
Number of gradient evaluations:	10
Number of hessian evaluations:	4
Cause of termination:	Relative gradient = 3.4e-07 <= 6.1e-06
Optimization time:	0:00:00.003244


Changing the requested precision


In [15]:
results = myBiogeme.\
estimate(algorithm=opt.newtonLineSearchForBiogeme,
         algoParameters={'tolerance': 0.1})
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144317,0.390742,0.369340,0.711874,0.365691,0.394641,0.693108
beta2,0.023428,0.036930,0.634377,0.525835,0.034256,0.683887,0.494047


In [16]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with line search
Relative gradient:	0.015380029144206175
Number of iterations:	2
Number of function evaluations:	7
Number of gradient evaluations:	7
Number of hessian evaluations:	3
Cause of termination:	Relative gradient = 0.015 <= 0.1
Optimization time:	0:00:00.001905


## Newton with trust region

In [17]:
results = myBiogeme.estimate(algorithm=opt.newtonTrustRegionForBiogeme)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685573,0.492982


In [18]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	3.289367618372832e-07
Cause of termination:	Relative gradient = 3.3e-07 <= 6.1e-06
Number of iterations:	3
Number of function evaluations:	6
Number of gradient evaluations:	4
Number of hessian evaluations:	4
Optimization time:	0:00:00.001830


We illustrate the parameters. We use the truncated conjugate gradient instead of dogleg for the trust region subproblem, starting with a small trust region of radius 0.001, and a maximum of 3 iterations.

In [19]:
results = myBiogeme.\
estimate(algorithm=opt.newtonTrustRegionForBiogeme,
         algoParameters={'dogleg':False,
                         'radius':0.001,
                         'maxiter':3})
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.000053,0.348093,0.000151,0.99988,0.308910,0.000170,0.999864
beta2,0.007000,0.032583,0.214830,0.82990,0.030019,0.233173,0.815627


In [20]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	2.0182963630997235
Cause of termination:	Maximum number of iterations reached: 3
Number of iterations:	3
Number of function evaluations:	6
Number of gradient evaluations:	4
Number of hessian evaluations:	4
Optimization time:	0:00:00.002176


Changing the requested precision


In [21]:
results = myBiogeme.\
estimate(algorithm=opt.newtonTrustRegionForBiogeme,
         algoParameters={'tolerance':0.1})
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144317,0.390742,0.369340,0.711874,0.365691,0.394641,0.693108
beta2,0.023428,0.036930,0.634377,0.525835,0.034256,0.683887,0.494047


In [22]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Unconstrained Newton with trust region
Relative gradient:	0.014746524905603029
Cause of termination:	Relative gradient = 0.015 <= 0.1
Number of iterations:	2
Number of function evaluations:	4
Number of gradient evaluations:	3
Number of hessian evaluations:	3
Optimization time:	0:00:00.001590


## BFGS with line search

In [23]:
results = myBiogeme.estimate(algorithm=opt.bfgsLineSearchForBiogeme)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685573,0.492982


In [24]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Inverse BFGS with line search
Relative gradient:	5.933646980468922e-07
Cause of termination:	Relative gradient = 5.9e-07 <= 6.1e-06
Number of iterations:	5
Number of function evaluations:	28
Number of gradient evaluations:	6
Optimization time:	0:00:00.003693


## BFGS with trust region

In [25]:
results = myBiogeme.estimate(algorithm=opt.bfgsTrustRegionForBiogeme)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144546,0.391018,0.369667,0.711631,0.366198,0.394721,0.693048
beta2,0.023502,0.036947,0.636087,0.524720,0.034280,0.685574,0.492982


In [26]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region
Relative gradient:	5.039394505504404e-07
Cause of termination:	Relative gradient = 5e-07 <= 6.1e-06
Number of iterations:	14
Number of function evaluations:	23
Number of gradient evaluations:	9
Optimization time:	0:00:00.007478


## Newton/BFGS with trust region for simple bounds

This is the default algorithm used by Biogeme. It is the implementation of the algorithm proposed by [Conn et al. (1988)](https://www.ams.org/journals/mcom/1988-50-182/S0025-5718-1988-0929544-3/S0025-5718-1988-0929544-3.pdf).

In [27]:
results = myBiogeme.\
estimate(algorithm=opt.simpleBoundsNewtonAlgorithmForBiogeme)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685573,0.492982


In [28]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Newton with trust region for simple bound constraints
Proportion analytical hessian:	100.0%
Relative projected gradient:	3.2893676168569793e-07
Relative change:	0.00022882036808896666
Number of iterations:	3
Number of function evaluations:	10
Number of gradient evaluations:	4
Number of hessian evaluations:	4
Cause of termination:	Relative gradient = 3.3e-07 <= 6.1e-06
Optimization time:	0:00:00.003137


When the second derivatives are too computationally expensive to calculate, it is possible to avoid calculating them at each successful iteration. The parameter 'proportionAnalyticalHessian' allows to control that.

In [29]:
results = myBiogeme.\
estimate(algorithm=opt.simpleBoundsNewtonAlgorithmForBiogeme,
         algoParameters = {'proportionAnalyticalHessian': 0.5})
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524721,0.034280,0.685573,0.492982


In [30]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Hybrid Newton [50.0%] with trust region for simple bound constraints
Proportion analytical hessian:	60.0%
Relative projected gradient:	2.790944329115531e-06
Relative change:	4.2680917955839615e-05
Number of iterations:	4
Number of function evaluations:	13
Number of gradient evaluations:	5
Number of hessian evaluations:	3
Cause of termination:	Relative gradient = 2.8e-06 <= 6.1e-06
Optimization time:	0:00:00.003492


If the parameter is set to zero, the second derivatives are not used at all, and the algorithm relies only on the BFGS update.

In [31]:
results = myBiogeme.\
estimate(algorithm=opt.simpleBoundsNewtonAlgorithmForBiogeme,
         algoParameters = {'proportionAnalyticalHessian': 0})
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144547,0.391018,0.369669,0.711629,0.366198,0.394723,0.693047
beta2,0.023502,0.036947,0.636088,0.524719,0.034280,0.685574,0.492981


In [32]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region for simple bound constraints
Proportion analytical hessian:	0.0%
Relative projected gradient:	1.3873770983933678e-06
Relative change:	6.637427884398361e-08
Number of iterations:	14
Number of function evaluations:	31
Number of gradient evaluations:	9
Number of hessian evaluations:	0
Cause of termination:	Relative change = 6.64e-08 <= 1e-05
Optimization time:	0:00:00.008565


There are shortcuts to call the BFGS and the Newton versions

In [33]:
results = myBiogeme.\
estimate(algorithm=opt.bioNewton)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144545,0.391017,0.369665,0.711632,0.366198,0.394720,0.693050
beta2,0.023502,0.036947,0.636086,0.524720,0.034280,0.685573,0.492982


In [34]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	Newton with trust region for simple bound constraints
Proportion analytical hessian:	100.0%
Relative projected gradient:	3.2893676168569793e-07
Relative change:	0.00022882036808896666
Number of iterations:	3
Number of function evaluations:	10
Number of gradient evaluations:	4
Number of hessian evaluations:	4
Cause of termination:	Relative gradient = 3.3e-07 <= 6.1e-06
Optimization time:	0:00:00.002522


In [35]:
results = myBiogeme.\
estimate(algorithm=opt.bioBfgs)
results.getEstimatedParameters()

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
beta1,0.144547,0.391018,0.369669,0.711629,0.366198,0.394723,0.693047
beta2,0.023502,0.036947,0.636088,0.524719,0.034280,0.685574,0.492981


In [36]:
for k, v in results.data.optimizationMessages.items():
    print(f'{k}:\t{v}')

Algorithm:	BFGS with trust region for simple bound constraints
Proportion analytical hessian:	0.0%
Relative projected gradient:	1.3873770983933678e-06
Relative change:	6.637427884398361e-08
Number of iterations:	14
Number of function evaluations:	31
Number of gradient evaluations:	9
Number of hessian evaluations:	0
Cause of termination:	Relative change = 6.64e-08 <= 1e-05
Optimization time:	0:00:00.008666
